In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
import time
import numpy as np
print(np.__version__)

1.16.2


In [2]:
import data_reader
SemData = data_reader.read_data_sets("data", padding=False)

TRAIN_FILE.txt existed
TEST_FILE.txt existed


In [3]:
print(SemData.train.sentences.shape)
print(SemData.test.sentences.shape)

(8000,)
(2717,)


In [5]:
class Params():
    def __init__(self):
        self.n_inputs = 300
        self.n_hidden = 150
        self.n_class = 19
        self.batch_size = 64
        self.n_train = 2000
        self.n_display = 100
        self.test_size = 2717

params = Params()

In [9]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.n_inputs = params.n_inputs
        self.n_hidden = params.n_hidden
        self.n_class = params.n_class
        self.batch_size = params.batch_size
        self.rnn = nn.RNN(self.n_inputs, self.n_hidden)
        self.fc = nn.Linear(self.n_hidden, self.n_class)
        self.h0 = torch.randn(1, 1, self.n_hidden).cuda()
    
    def forward(self, sentences):
        for i in range(len(sentences)):
            sen_len = sentences[i].shape[0]
            sentence = sentences[i].reshape(1, sen_len, self.n_inputs)
            sentence = torch.tensor(sentence.transpose(1, 0, 2)).cuda()
            _, h = self.rnn(sentence, self.h0)
            h = torch.nn.functional.dropout(h, 0.5)
            out = self.fc(h[0])
            if i==0:
                output = out
            else:
                output = torch.cat([output, out])
        return output

In [10]:
model = RNN()
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
def train():
    start = time.clock()
    test_sentences = SemData.test.sentences
    test_labels = torch.tensor(SemData.test.labels).cuda()
    for step in range(params.n_train):
        sentences, labels = SemData.train.next_batch(params.batch_size)
        labels = torch.tensor(labels).cuda()
        optimizer.zero_grad()
        logits = model(sentences)
        _, train_golden = torch.max(labels, 1)
        loss = criterion(logits, train_golden)
        loss.backward()
        optimizer.step()

        if step % params.n_display == 0:
            _, preds = torch.max(logits, 1)
            train_accuracy = torch.sum(preds == train_golden).item() / params.batch_size
            
            test_logits = model(test_sentences)
            _, test_preds = torch.max(test_logits, 1)
            _, test_golden = torch.max(test_labels, 1)
            test_loss = criterion(test_logits, test_golden)
            test_accuracy = torch.sum(test_preds == test_golden).item() / params.test_size
            print("step: %d" % (step))
            print("train_accuracy: %.4g %% local_loss: %.8g" % (train_accuracy*100, loss.item()))
            print("test_accuracy: %.4g %% total_loss: %.8g" % (test_accuracy*100, test_loss.item()))
    print("------------------------------------")
    print("training time: ", time.clock()-start, " s")
    test_logits = model(test_sentences)
    _, test_preds = torch.max(test_logits, 1)
    _, test_golden = torch.max(test_labels, 1)
    test_loss = criterion(test_logits, test_golden)
    test_accuracy = torch.sum(test_preds == test_golden).item() / params.test_size
    print("test_accuracy: %.4g %% total_loss: %.8g" % (test_accuracy*100, test_loss.item()))

In [12]:
model = train()

train_step:     0
train_accuracy: 3.125 % local_loss: 2.9850862
test_accuracy: 7.6 % total_loss: 2.9413893
train_step:   100
train_accuracy: 10.94 % local_loss: 2.7304223
test_accuracy: 14.4 % total_loss: 2.7068486
train_step:   200
train_accuracy: 17.19 % local_loss: 2.578572
test_accuracy: 17 % total_loss: 2.6576321
train_step:   300
train_accuracy: 20.31 % local_loss: 2.519654
test_accuracy: 16 % total_loss: 2.6906371
train_step:   400
train_accuracy: 26.56 % local_loss: 2.480572
test_accuracy: 14.8 % total_loss: 2.6859848
train_step:   500
train_accuracy: 21.88 % local_loss: 2.2596779
test_accuracy: 16.8 % total_loss: 2.8894606
train_step:   600
train_accuracy: 40.62 % local_loss: 2.0222893
test_accuracy: 16.6 % total_loss: 2.7029908
train_step:   700
train_accuracy: 46.88 % local_loss: 1.8059374
test_accuracy: 17.4 % total_loss: 2.7462423
train_step:   800
train_accuracy: 46.88 % local_loss: 1.9547479
test_accuracy: 17.4 % total_loss: 2.7834785
train_step:   900
train_accuracy: 51

RuntimeError: Expected object of scalar type Long but got scalar type Double for argument #2 'target'